In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [3]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); 
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); 
ins_24_26 = pd.read_csv( loc_ts + "\\ins_24_26.csv" ); 

# Genero los Sets con el primer install

In [4]:
ins_18_20["date"] =  pd.to_datetime( ins_18_20["date"] ); ins_18_20["_sc"] = ins_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); ins_18_20["_sc"] = ins_18_20["_sc"].dt.total_seconds();
ins_21_23["date"] =  pd.to_datetime( ins_21_23["date"] ); ins_21_23["_sc"] = ins_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); ins_21_23["_sc"] = ins_21_23["_sc"].dt.total_seconds(); 
ins_24_26["date"] =  pd.to_datetime( ins_24_26["date"] ); ins_24_26["_sc"] = ins_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); ins_24_26["_sc"] = ins_24_26["_sc"].dt.total_seconds(); 

In [5]:
ins_18_20_first_ins = ins_18_20.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_21_23_first_ins = ins_21_23.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_24_26_first_ins = ins_24_26.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

# Sacamos los Features

In [6]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_ins"

In [7]:
rh_18_20 = ins_18_20_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = ins_21_23_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = ins_18_20_first_ins[ ["_sc"] ]
tg_21_23 = ins_21_23_first_ins[ ["_sc"] ]

## <span style="color:yellow"> ============================================================================================================================ </span> 

## <span style="color:green"> **Hora de la conversion** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. Por ejemplo que las 00 hs tengan un valor similar a 23

In [ ]:
hour_18_20 = rh_18_20.copy()
hour_21_23 = rh_21_23.copy()

hour_18_20["sin_hour"] = pd.to_datetime( ins_18_20_first_ins["date"] ).dt.hour; hour_18_20["sin_hour"] = hour_18_20["sin_hour"].apply( lambda x: np.sin( (x *np.pi)/24 ) )
hour_21_23["sin_hour"] = pd.to_datetime( ins_21_23_first_ins["date"] ).dt.hour; hour_21_23["sin_hour"] = hour_21_23["sin_hour"].apply( lambda x: np.sin( (x *np.pi)/24 ) )

hour_18_20.to_csv( loc_ftr + "\\sin_hour_18_20.csv", index = False )
hour_21_23.to_csv( loc_ftr + "\\sin_hour_21_23.csv", index = False )

## <span style="color:green"> **Dia de la conversion** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. Por ejemplo que las 00 hs tengan un valor similar a 23

In [206]:
week_day_18_20 = rh_18_20.copy()
week_day_21_23 = rh_21_23.copy()

week_day_18_20["sin_week_day"] = pd.to_datetime( ins_18_20_first_ins["date"] ).dt.weekday; week_day_18_20["sin_week_day"] = week_day_18_20["sin_week_day"].apply( lambda x: np.sin( (x * np.pi) / 6 ) )
week_day_21_23["sin_week_day"] = pd.to_datetime( ins_21_23_first_ins["date"] ).dt.weekday; week_day_21_23["sin_week_day"] = week_day_21_23["sin_week_day"].apply( lambda x: np.sin( (x * np.pi) / 6 ) )

week_day_18_20.to_csv( loc_ftr + "\\week_day_18_20.csv", index = False )
week_day_21_23.to_csv( loc_ftr + "\\week_day_21_23.csv", index = False )

## <span style="color:green"> **Hora mas popular del usuario en las instalaciones** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [17]:
main_ihr_18_20 = rh_18_20.copy()
main_ihr_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20[["ref_hash"]]; hour_mode_18_20["hour"] = pd.to_datetime( ins_18_20["date"] ).dt.hour
hour_mode_21_23 = ins_21_23[["ref_hash"]]; hour_mode_21_23["hour"] = pd.to_datetime( ins_21_23["date"] ).dt.hour

hour_mode_18_20 = hour_mode_18_20.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_18_20 = hour_mode_18_20.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)

main_ihr_18_20 = main_ihr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ihr_21_23 = main_ihr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ihr_18_20["main_hour"] =  main_ihr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ihr_21_23["main_hour"] =  main_ihr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ihr_18_20.to_csv( loc_ftr + "\\main_ihr_18_20.csv", index = False )
main_ihr_21_23.to_csv( loc_ftr + "\\main_ihr_21_23.csv", index = False )

## <span style="color:red"> **Cantidad de instalaciones por dispositivo en la ventana previa a la conversion** </span> 

In [51]:
cant_ins_18_20 = rh_18_20.copy()
cant_ins_21_23 = rh_21_23.copy()

cant_ins_18_20 = cant_ins_18_20.merge( ins_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_ins_21_23 = cant_ins_21_23.merge( ins_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_ins_18_20.to_csv( loc_ftr + "\\cant_ins_18_20.csv", index = False )
cant_ins_21_23.to_csv( loc_ftr + "\\cant_ins_21_23.csv", index = False )

## <span style="color:red"> **IP del dispositivo al realizar la primera instalacion** </span> 

In [114]:
ip_encod_18_20 = rh_18_20.copy()
ip_encod_21_23 = rh_21_23.copy()

ip_encod_18_20["ip_address"] = ins_18_20_first_ins["ip_address"];
ip_encod_21_23["ip_address"] = ins_21_23_first_ins["ip_address"];

appces_18_20_ins = ins_18_20["ip_address"].value_counts().reset_index().rename( columns = {"index":"ip_address", "ip_address":"appearances"} )
appces_21_23_ins = ins_21_23["ip_address"].value_counts().reset_index().rename( columns = {"index":"ip_address", "ip_address":"appearances"} )

appces_18_20_ins["ip_address_mean"] = appces_18_20_ins["appearances"] / len( ins_18_20 ); appces_18_20_ins.drop( "appearances", axis = 1, inplace = True )
appces_21_23_ins["ip_address_mean"] = appces_21_23_ins["appearances"] / len( ins_21_23 ); appces_21_23_ins.drop( "appearances", axis = 1, inplace = True )

ip_encod_18_20 = ip_encod_18_20.merge( appces_18_20_ins, how = "left", on = "ip_address" ).drop( "ip_address", axis = 1 )
ip_encod_21_23 = ip_encod_21_23.merge( appces_21_23_ins, how = "left", on = "ip_address" ).drop( "ip_address", axis = 1 )

ip_encod_18_20.to_csv( loc_ftr + "\\ip_encod_18_20.csv", index = False )
ip_encod_21_23.to_csv( loc_ftr + "\\ip_encod_21_23.csv", index = False )

## <span style="color:green"> **WiFi en la instalacion** </span>

#### <span style="color:orange"> **Mean Encoding** </span> 

In [26]:
wifi_ins_18_20 = rh_18_20.copy()
wifi_ins_21_23 = rh_21_23.copy()

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

wf_18_20_sc = ins_18_20_first_ins[ ["ref_hash","wifi"] ].merge( sc, how = "left", on = "ref_hash" ); 

X_wf = wf_18_20_sc[ ["wifi"] ].astype(str)
Y_wf = wf_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_wf, Y_wf );

wifi_ins_18_20["wifi"] = te.transform( ins_18_20_first_ins[ ["wifi"] ].astype( str ) );
wifi_ins_21_23["wifi"] = te.transform( ins_21_23_first_ins[ ["wifi"] ].astype( str ) ); 

wifi_ins_18_20.to_csv( loc_ftr + "\\wifi_ins_18_20.csv", index = False )
wifi_ins_21_23.to_csv( loc_ftr + "\\wifi_ins_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> (Usando el SC para el promedio) - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [21]:
wifi_ins_18_20 = rh_18_20.copy()
wifi_ins_21_23 = rh_21_23.copy()

wf_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["wifi"] ] )
wf_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["wifi"] ] )

wifi_ins_18_20 = pd.concat( [wifi_ins_18_20, wf_ohe_18_20], axis = 1 )
wifi_ins_21_23 = pd.concat( [wifi_ins_21_23, wf_ohe_21_23], axis = 1 )

wifi_ins_18_20.to_csv( loc_ftr + "\\wifi_ins_18_20.csv", index = False )
wifi_ins_21_23.to_csv( loc_ftr + "\\wifi_ins_21_23.csv", index = False )

## <span style="color:green"> **Aplicacion mas instalada por el usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [19]:
main_app_18_20 = rh_18_20.copy()
main_app_21_23 = rh_21_23.copy()

app_mas_ins_18_20 = ins_18_20.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()
app_mas_ins_21_23 = ins_21_23.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()

app_mas_ins_18_20 = app_mas_ins_18_20.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_18_20["cant_ins"] )
app_mas_ins_21_23 = app_mas_ins_21_23.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_21_23["cant_ins"] ) 

main_app_18_20 = main_app_18_20.merge( app_mas_ins_18_20, how = "left", on = "ref_hash" )
main_app_21_23 = main_app_21_23.merge( app_mas_ins_21_23, how = "left", on = "ref_hash" )

main_app_18_20["to_count"] = 1; main_app_18_20["application_id"] = main_app_18_20[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_18_20); del(main_app_18_20["to_count"])
main_app_21_23["to_count"] = 1; main_app_21_23["application_id"] = main_app_21_23[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_21_23); del(main_app_21_23["to_count"])

main_app_18_20.to_csv( loc_ftr + "\\main_app_18_20.csv", index = False )
main_app_21_23.to_csv( loc_ftr + "\\main_app_21_23.csv", index = False )

## <span style="color:red"> **Cantidad de instalaciones implicitas por dispositivo** </span> 

In [873]:
cins_imp_18_20 = rh_18_20.copy()
cins_imp_21_23 = rh_21_23.copy()

cins_imp_18_20["cant_imp"] = ins_18_20["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_18_20 = cins_imp_18_20.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()
cins_imp_21_23["cant_imp"] = ins_21_23["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_21_23 = cins_imp_21_23.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()

cins_imp_18_20.to_csv( loc_ftr + "\\cins_imp_18_20.csv", index = False )
cins_imp_21_23.to_csv( loc_ftr + "\\cins_imp_21_23.csv", index = False )

## <span style="color:red"> **Tipo de la primera instalacion (implicita/no implicita)** </span> 

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [30]:
type_ins_18_20 = rh_18_20.copy()
type_ins_21_23 = rh_21_23.copy()

type_ins_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["implicit"] ].astype( "category" ) )
type_ins_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["implicit"] ].astype( "category" ) )

type_ins_18_20 = pd.concat( [type_ins_18_20, type_ins_ohe_18_20], axis = 1 )
type_ins_21_23 = pd.concat( [type_ins_21_23, type_ins_ohe_21_23], axis = 1 )

type_ins_18_20.to_csv( loc_ftr + "\\type_ins_18_20.csv", index = False )
type_ins_21_23.to_csv( loc_ftr + "\\type_ins_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones atribuidas por dispositivo** </span> 

In [9]:
cins_atr_18_20 = rh_18_20.copy()
cins_atr_21_23 = rh_21_23.copy()

cins_atr_18_20["cant_atr"] = ins_18_20["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_18_20 = cins_atr_18_20.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()
cins_atr_21_23["cant_atr"] = ins_21_23["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_21_23 = cins_atr_21_23.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()

cins_atr_18_20.to_csv( loc_ftr + "\\cins_atr_18_20.csv", index = False )
cins_atr_21_23.to_csv( loc_ftr + "\\cins_atr_21_23.csv", index = False )

## <span style="color:green"> **Ref Type del dispositivo que realizo la instalacion** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> 

In [ ]:
ref_type_18_20 = rh_18_20.copy()
ref_type_21_23 = rh_21_23.copy()

rt_18_20_sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_type","_sc"] ]

rt_18_20_sc = ins_18_20_first_ins[ ["ref_hash","ref_type"] ].merge( sc, how = "left", on = "ref_hash" ); rl_18_20_sc.fillna( 259200, inplace = True )

X_rt = dl_18_20_sc[ ["ref_type"] ]
Y_rt = dl_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True)
te.fit_transform( X_rt, Y_rt );

ref_type_18_20["ref_type_enc"] = te.transform( ins_18_20_first_ins[["ref_type"]].astype(str) )
ref_type_21_23["ref_type_enc"] = te.transform( ins_21_23_first_ins[["ref_type"]].astype(str) )

ref_type_18_20.to_csv( loc_ftr + "\\ref_type_18_20.csv", index = False )
ref_type_21_23.to_csv( loc_ftr + "\\ref_type_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [36]:
ref_type_18_20 = rh_18_20
ref_type_21_23 = rh_21_23

rt_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["ref_type"] ].astype("category") )
rt_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["ref_type"] ].astype("category") )

ref_type_18_20 = pd.concat( [ref_type_18_20, rt_ohe_18_20], axis = 1 )
ref_type_21_23 = pd.concat( [ref_type_21_23, rt_ohe_21_23], axis = 1 )

ref_type_18_20.to_csv( loc_ftr + "\\ref_type_18_20.csv", index = False )
ref_type_21_23.to_csv( loc_ftr + "\\ref_type_21_23.csv", index = False )

## <span style="color:green"> **DeviceLenguage del dispositivo al realizar la instlacion** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el SC para el promedio)

In [39]:
dev_lang_18_20 = rh_18_20.copy()
dev_lang_21_23 = rh_21_23.copy()

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

dl_18_20_sc = ins_18_20_first_ins[ ["ref_hash","device_language"] ].merge( sc, how = "left", on = "ref_hash" ); dl_18_20_sc.fillna( 259200, inplace = True )

X_dl = dl_18_20_sc[ ["device_language"] ]
Y_dl = dl_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_dl, Y_dl );

# replace nans with the global mean ( https://necromuralist.github.io/kaggle-competitions/posts/mean-encoding/ )
global_mean = Y_dl["_sc"].mean()

dev_lang_18_20["dev_language"] = te.transform( ins_18_20_first_ins[ ["device_language"] ].astype( str ) ); dev_lang_18_20["dev_language"].replace( {"nan":global_mean}, inplace = True )
dev_lang_21_23["dev_language"] = te.transform( ins_21_23_first_ins[ ["device_language"] ].astype( str ) ); dev_lang_21_23["dev_language"].replace( {"nan":global_mean}, inplace = True )

dev_lang_18_20.to_csv( loc_ftr + "\\dev_lang_18_20.csv", index = False )
dev_lang_21_23.to_csv( loc_ftr + "\\dev_lang_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span>

In [40]:
dev_lang_18_20 = rh_18_20.copy()
dev_lang_21_23 = rh_21_23.copy()

#Generamos todas las categorias posibles para que tengan las mismas columnas
dl_18_20 = ins_18_20_first_ins["device_language"].drop_duplicates()
dl_21_23 = ins_21_23_first_ins["device_language"].drop_duplicates()

#Creamos la categoria. Los NAN son una categoria
dl_category = pd.api.types.CategoricalDtype( dl_18_20.append( dl_21_23 ).fillna("NoDL").drop_duplicates() ) 

dl_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["device_language"] ].astype(dl_category) )
dl_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["device_language"] ].astype(dl_category) )

dev_lang_18_20 = pd.concat( [dev_lang_18_20, dl_ohe_18_20], axis = 1 )
dev_lang_21_23 = pd.concat( [dev_lang_21_23, dl_ohe_21_23], axis = 1 )

dev_lang_18_20.to_csv( loc_ftr + "\\dev_lang_18_20.csv", index = False )
dev_lang_21_23.to_csv( loc_ftr + "\\dev_lang_21_23.csv", index = False )

## <span style="color:green"> **Modelo del dispositivo uilizado** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el SC para el promedio) 

In [41]:
dev_modl_18_20 = rh_18_20.copy()
dev_modl_21_23 = rh_21_23.copy()

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

dm_18_20_sc = ins_18_20_first_ins[ ["ref_hash","device_model"] ].merge( sc, how = "left", on = "ref_hash" ); 

X_dm = dm_18_20_sc[ ["device_model"] ].astype(str)
Y_dm = dm_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_dm, Y_dm );

# replace nans with the global mean ( https://necromuralist.github.io/kaggle-competitions/posts/mean-encoding/ )
global_mean = Y_dm["_sc"].mean()

dev_modl_18_20["device_model"] = te.transform( ins_18_20_first_ins[ ["device_model"] ].astype( str ) ); dev_modl_18_20.fillna( global_mean, inplace = True )
dev_modl_21_23["device_model"] = te.transform( ins_21_23_first_ins[ ["device_model"] ].astype( str ) ); dev_modl_21_23.fillna( global_mean, inplace = True )

dev_modl_18_20.to_csv( loc_ftr + "\\dev_modl_18_20.csv", index = False )
dev_modl_21_23.to_csv( loc_ftr + "\\dev_modl_21_23.csv", index = False )

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el promedio de cuantas veces fue utilizado) 

In [42]:
dev_modl_18_20 = rh_18_20.copy()
dev_modl_21_23 = rh_21_23.copy()

dev_modl_mean_18_20 = ins_18_20[ ["device_model","ref_hash"] ].groupby("device_model").agg("count"); dev_modl_mean_18_20["mean"] = dev_modl_mean_18_20["ref_hash"]/len(ins_18_20); del(dev_modl_mean_18_20["ref_hash"])
dev_modl_mean_21_23 = ins_21_23[ ["device_model","ref_hash"] ].groupby("device_model").agg("count"); dev_modl_mean_21_23["mean"] = dev_modl_mean_21_23["ref_hash"]/len(ins_21_23); del(dev_modl_mean_21_23["ref_hash"])

dev_modl_18_20["device_model"] = ins_18_20_first_ins["device_model"]; dev_modl_18_20 = dev_modl_18_20.merge(dev_modl_mean_18_20, how = "left", on = "device_model" )
dev_modl_21_23["device_model"] = ins_21_23_first_ins["device_model"]; dev_modl_21_23 = dev_modl_21_23.merge(dev_modl_mean_21_23, how = "left", on = "device_model" )

#Contamos a los NaNs como un "modelo"
mean_nan_18_20 = ins_18_20["device_model"].isnull().sum() / len( ins_18_20 )
mean_nan_21_23 = ins_21_23["device_model"].isnull().sum() / len( ins_21_23 )

dev_modl_18_20.fillna( mean_nan_18_20, inplace = True ); del( dev_modl_18_20["device_model"] ); dev_modl_18_20.rename( {"mean":"dev_mod"}, inplace = True )
dev_modl_21_23.fillna( mean_nan_21_23, inplace = True ); del( dev_modl_21_23["device_model"] ); dev_modl_21_23.rename( {"mean":"dev_mod"}, inplace = True )

dev_modl_18_20.to_csv( loc_ftr + "\\dev_modl_18_20.csv", index = False )
dev_modl_21_23.to_csv( loc_ftr + "\\dev_modl_21_23.csv", index = False )

## <span style="color:red"> **SessionUserAgent de los dispositivos** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el SC para el promedio)

In [43]:
ins_dsua_18_20 = rh_18_20.copy()
ins_dsua_21_23 = rh_21_23.copy()

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

sua_18_20_sc = ins_18_20_first_ins[ ["ref_hash","session_user_agent"] ].merge( sc, how = "left", on = "ref_hash" ); 

X_sua = sua_18_20_sc[ ["session_user_agent"] ].astype(str)
Y_sua = sua_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_sua, Y_sua );

ins_dsua_18_20["sua"] = te.transform( ins_18_20_first_ins[ ["session_user_agent"] ].astype( str ) );
ins_dsua_21_23["sua"] = te.transform( ins_21_23_first_ins[ ["session_user_agent"] ].astype( str ) ); 

ins_dsua_18_20.fillna( Y_sua["_sc"].mean(), inplace = True )
ins_dsua_21_23.fillna( Y_sua["_sc"].mean(), inplace = True )

ins_dsua_18_20.to_csv( loc_ftr + "\\ins_dsua_18_20.csv", index = False )
ins_dsua_21_23.to_csv( loc_ftr + "\\ins_dsua_21_23.csv", index = False )

## <span style="color:green"> **Ref Hash** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de apariciones en las instalaciones en la ventana anterior)

In [ ]:
rh_encod_18_20 = rh_18_20.copy()
rh_encod_21_23 = rh_21_23.copy()

appces_18_20_ins = ins_18_20["ref_hash"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "ref_hash":"appearances"} )
appces_21_23_ins = ins_21_23["ref_hash"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "ref_hash":"appearances"} )

appces_18_20_ins["ref_hash_mean"] = appces_18_20_ins["appearances"] / len( ins_18_20 ); appces_18_20_ins.drop( "appearances", axis = 1, inplace = True )
appces_21_23_ins["ref_hash_mean"] = appces_21_23_ins["appearances"] / len( ins_21_23 ); appces_21_23_ins.drop( "appearances", axis = 1, inplace = True )

rh_encod_18_20 = rh_encod_18_20.merge( appces_18_20_ins, how = "left", on = "ref_hash" )
rh_encod_21_23 = rh_encod_21_23.merge( appces_21_23_ins, how = "left", on = "ref_hash" )

rh_encod_18_20.to_csv( loc_ftr + "\\rh_encod_18_20.csv", index = False )
rh_encod_21_23.to_csv( loc_ftr + "\\rh_encod_21_23.csv", index = False )

Son muchos NaNs, por lo que no se pueden completar con nada coherente. Esto agrega demasiado ruido al set de entrenamiento

## <span style="color:red"> **Tipo de la primera instalacion (Atribuida/No Atribuida)** </span>

#### <span style="color:orange"> **OneHotEncoding** </span> - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [47]:
atbt_ins_18_20 = rh_18_20.copy()
atbt_ins_21_23 = rh_21_23.copy()

atbt_ins_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["attributed"] ].astype( str ) )
atbt_ins_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["attributed"] ].astype( str ) )

atbt_ins_ohe_18_20 = pd.concat( [atbt_ins_18_20, atbt_ins_ohe_18_20], axis = 1 )
atbt_ins_ohe_21_23 = pd.concat( [atbt_ins_21_23, atbt_ins_ohe_21_23], axis = 1 )

atbt_ins_18_20.to_csv( loc_ftr + "\\atbt_ins_18_20.csv", index = False )
atbt_ins_21_23.to_csv( loc_ftr + "\\atbt_ins_21_23.csv", index = False )

#### <span style="color:orange"> **Mean Encoding** </span> (Usando como promedio la cantidad de veces que la primera instalacion fue de ese determinado tipo sobre el total de instalaciones)

In [72]:
atbt_ins_18_20 = rh_18_20.copy()
atbt_ins_21_23 = rh_21_23.copy()

atbt_ins_18_20["atbt"] = ins_18_20_first_ins["attributed"]; atbt_ins_18_20["cant_ins"] = atbt_ins_18_20.groupby( "atbt" ).transform( "count" ); atbt_ins_18_20["atbt_mean"] = atbt_ins_18_20["cant_ins"]/len(rh_18_20)
atbt_ins_21_23["atbt"] = ins_21_23_first_ins["attributed"]; atbt_ins_21_23["cant_ins"] = atbt_ins_21_23.groupby( "atbt" ).transform( "count" ); atbt_ins_21_23["atbt_mean"] = atbt_ins_21_23["cant_ins"]/len(rh_21_23)

atbt_ins_18_20.drop( ["atbt","cant_ins"], axis = 1, inplace = True )
atbt_ins_21_23.drop( ["atbt","cant_ins"], axis = 1, inplace = True )

atbt_ins_18_20.to_csv( loc_ftr + "\\atbt_ins_18_20.csv", index = False )
atbt_ins_21_23.to_csv( loc_ftr + "\\atbt_ins_21_23.csv", index = False )

## <span style="color:red"> **Tiempo hasta la primer instlacion en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer instlacion

In [87]:
frst_ins_18_20 = ins_18_20_first_ins[ ["ref_hash", "_sc"] ]
frst_ins_21_23 = ins_21_23_first_ins[ ["ref_hash", "_sc"] ]

frst_ins_18_20.rename( columns = {"_sc": "time_to_ins"}, inplace = True )
frst_ins_21_23.rename( columns = {"_sc": "time_to_ins"}, inplace = True )

frst_ins_18_20.to_csv( loc_ftr + "\\frst_ins_18_20.csv", index = False )
frst_ins_21_23.to_csv( loc_ftr + "\\frst_ins_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultima instalacion en esa ventana** </span> 

In [56]:
last_ins_18_20 = rh_18_20.copy()
last_ins_21_23 = rh_21_23.copy()

last_installs_rh_ins_18_20 = ins_18_20[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_installs_rh_ins_21_23 = ins_21_23[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_installs_rh_ins_18_20["time_to_lst_ins"] = ( pd.to_datetime( last_installs_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_installs_rh_ins_21_23["time_to_lst_ins"] = ( pd.to_datetime( last_installs_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_installs_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
last_installs_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

last_ins_18_20 = last_ins_18_20.merge( last_installs_rh_ins_18_20, how = "left", on = "ref_hash" )
last_ins_21_23 = last_ins_21_23.merge( last_installs_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_ins_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_ins_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_ins_18_20.to_csv( loc_ftr + "\\last_ins_18_20.csv", index = False )
last_ins_21_23.to_csv( loc_ftr + "\\last_ins_21_23.csv", index = False )

## <span style="color:red"> **Instalo entre 21 hs y 3 hs (Noche)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [38]:
ins_18_20['ins_21_3'] = (ins_18_20['date'].dt.hour > 20) | (ins_18_20['date'].dt.hour < 4) 
ins_21_23['ins_21_3'] = (ins_21_23['date'].dt.hour > 20) | (ins_21_23['date'].dt.hour < 4)

ins_night_18_20 = rh_18_20.copy()
ins_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()

ins_night_18_20 = ins_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_night_21_23 = ins_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_night_18_20["ins_21_3"] =  (ins_night_18_20["ins_21_3"] > 1).astype('int8')
ins_night_21_23["ins_21_3"] =  (ins_night_21_23["ins_21_3"] > 1).astype('int8')

ins_night_18_20.to_csv( loc_ftr + "\\ins_ngoh_18_20.csv", index = False )
ins_night_21_23.to_csv( loc_ftr + "\\ins_ngoh_21_23.csv", index = False )

del ins_18_20['ins_21_3']
del ins_21_23['ins_21_3']

## <span style="color:red"> **Instalo entre 4 hs y 10 hs (Mañana)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [40]:
ins_18_20['ins_4_10'] = (ins_18_20['date'].dt.hour > 3) & (ins_18_20['date'].dt.hour < 11) 
ins_21_23['ins_4_10'] = (ins_21_23['date'].dt.hour > 3) & (ins_21_23['date'].dt.hour < 11)

ins_morn_18_20 = rh_18_20.copy()
ins_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()

ins_morn_18_20 = ins_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_morn_21_23 = ins_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_morn_18_20["ins_4_10"] =  (ins_morn_18_20["ins_4_10"] > 1).astype('int8')
ins_morn_21_23["ins_4_10"] =  (ins_morn_21_23["ins_4_10"] > 1).astype('int8')

ins_morn_18_20.to_csv( loc_ftr + "\\ins_mroh_18_20.csv", index = False )
ins_morn_21_23.to_csv( loc_ftr + "\\ins_mroh_21_23.csv", index = False )

del ins_18_20['ins_4_10']
del ins_21_23['ins_4_10']

## <span style="color:red"> **Instalo entre 11 hs y 15 hs (Medio dia)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [42]:
ins_18_20['ins_11_15'] = (ins_18_20['date'].dt.hour > 10) & (ins_18_20['date'].dt.hour < 16) 
ins_21_23['ins_11_15'] = (ins_21_23['date'].dt.hour > 10) & (ins_21_23['date'].dt.hour < 16)

ins_midday_18_20 = rh_18_20.copy()
ins_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()

ins_midday_18_20 = ins_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_midday_21_23 = ins_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_midday_18_20["ins_11_15"] =  (ins_midday_18_20["ins_11_15"] > 1).astype('int8')
ins_midday_21_23["ins_11_15"] =  (ins_midday_21_23["ins_11_15"] > 1).astype('int8')

ins_midday_18_20.to_csv( loc_ftr + "\\ins_mdoh_18_20.csv", index = False )
ins_midday_21_23.to_csv( loc_ftr + "\\ins_mdoh_21_23.csv", index = False )

del ins_18_20['ins_11_15']
del ins_21_23['ins_11_15']

## <span style="color:red"> **Instalo entre 16 hs y 20 hs (Tarde)** </span> 

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [44]:
ins_18_20['ins_16_20'] = (ins_18_20['date'].dt.hour > 15) & (ins_18_20['date'].dt.hour < 21) 
ins_21_23['ins_16_20'] = (ins_21_23['date'].dt.hour > 15) & (ins_21_23['date'].dt.hour < 21)

ins_after_18_20 = rh_18_20.copy()
ins_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()

ins_after_18_20 = ins_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_after_21_23 = ins_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_after_18_20["ins_16_20"] =  (ins_after_18_20["ins_16_20"] > 1).astype('int8')
ins_after_21_23["ins_16_20"] =  (ins_after_21_23["ins_16_20"] > 1).astype('int8')

ins_after_18_20.to_csv( loc_ftr + "\\ins_afoh_18_20.csv", index = False )
ins_after_21_23.to_csv( loc_ftr + "\\ins_afoh_21_23.csv", index = False )

del ins_18_20['ins_16_20']
del ins_21_23['ins_16_20']

## <span style="color:green"> **"Kind" de la primera instalacion** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> - (Usando el promedio de cuantas intalaciones en total se realizaron de ese "kind") 

In [46]:
kind_ins_18_20 = rh_18_20.copy()
kind_ins_21_23 = rh_21_23.copy()

In [47]:
kind_ins_18_20 = kind_ins_18_20.merge( ins_18_20_first_ins[ ["ref_hash", "kind"] ], how = "left", on = "ref_hash" );
kind_ins_21_23 = kind_ins_21_23.merge( ins_21_23_first_ins[ ["ref_hash", "kind"] ], how = "left", on = "ref_hash" );

In [48]:
kind_count_18_20 = ins_18_20["kind"].value_counts().to_frame().reset_index().rename( columns = {"index":"kind", "kind":"count"} )
kind_count_21_23 = ins_21_23["kind"].value_counts().to_frame().reset_index().rename( columns = {"index":"kind", "kind":"count"} )

In [49]:
kind_ins_18_20 = kind_ins_18_20.merge( kind_count_18_20, how = "left", on = "kind" ); kind_ins_18_20["count"] = kind_ins_18_20["count"] / len(ins_18_20); del( kind_ins_18_20["kind"] )
kind_ins_21_23 = kind_ins_21_23.merge( kind_count_21_23, how = "left", on = "kind" ); kind_ins_21_23["count"] = kind_ins_21_23["count"] / len(ins_21_23); del( kind_ins_21_23["kind"] )

In [50]:
nan_18_20 = kind_ins_18_20["count"].isnull().sum() / len(ins_18_20); kind_ins_18_20.fillna( nan_18_20, inplace = True )
nan_21_23 = kind_ins_21_23["count"].isnull().sum() / len(ins_21_23); kind_ins_21_23.fillna( nan_21_23, inplace = True )

In [52]:
kind_ins_18_20.to_csv( loc_ftr + "\\kind_ins_18_20.csv", index = False )
kind_ins_21_23.to_csv( loc_ftr + "\\kind_ins_21_23.csv", index = False )

## <span style="color:green"> **Promedio del tiempo de instalacion** </span> 

In [65]:
tins_prm_18_20 = rh_18_20.copy()
tins_prm_21_23 = rh_21_23.copy()

In [66]:
ins_18_20["time_to_ins"] = (pd.to_datetime( ins_18_20["date"] ) - dt.datetime( 2019, 4, 18 )).dt.total_seconds()
ins_21_23["time_to_ins"] = (pd.to_datetime( ins_21_23["date"] ) - dt.datetime( 2019, 4, 21 )).dt.total_seconds()

In [67]:
time_to_ins_mean_18_20 = ins_18_20[ ["ref_hash","time_to_ins"] ].groupby( "ref_hash" ).mean()
time_to_ins_mean_21_23 = ins_21_23[ ["ref_hash","time_to_ins"] ].groupby( "ref_hash" ).mean()

In [68]:
tins_prm_18_20 = tins_prm_18_20.merge( time_to_ins_mean_18_20, how = "left", on = "ref_hash" )
tins_prm_21_23 = tins_prm_21_23.merge( time_to_ins_mean_21_23, how = "left", on = "ref_hash" )

In [70]:
tins_prm_18_20.to_csv( loc_ftr + "\\tins_prm_18_20.csv", index = False )
tins_prm_21_23.to_csv( loc_ftr + "\\tins_prm_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta que el usuario realizo alguna instalacion de un app del TOP 3** </span>

In [71]:
tt_3_app_18_20 = rh_18_20.copy()
tt_3_app_21_23 = rh_21_23.copy()

cond_1_18_20 = ins_18_20['application_id'] == ins_18_20['application_id'].value_counts().head().index[0]; cond_1_21_23 = ins_21_23['application_id'] == ins_21_23['application_id'].value_counts().head().index[0];
cond_2_18_20 = ins_18_20['application_id'] == ins_18_20['application_id'].value_counts().head().index[1]; cond_2_21_23 = ins_21_23['application_id'] == ins_21_23['application_id'].value_counts().head().index[1];
cond_3_18_20 = ins_18_20['application_id'] == ins_18_20['application_id'].value_counts().head().index[2]; cond_3_21_23 = ins_21_23['application_id'] == ins_21_23['application_id'].value_counts().head().index[2];

m1 = ins_18_20.loc[ cond_1_18_20 | cond_2_18_20 | cond_3_18_20, ["ref_hash", "date"] ].sort_values( by = "date", ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
m2 = ins_21_23.loc[ cond_1_21_23 | cond_2_21_23 | cond_3_21_23, ["ref_hash", "date"] ].sort_values( by = "date", ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

tt_3_app_18_20 = tt_3_app_18_20.merge( m1, how = "left", on = "ref_hash" )
tt_3_app_21_23 = tt_3_app_21_23.merge( m2, how = "left", on = "ref_hash" )

tt_3_app_18_20["date"] = ( pd.to_datetime( tt_3_app_18_20["date"] ) - dt.datetime( 2019, 4, 18 ) ).dt.total_seconds()
tt_3_app_21_23["date"] = ( pd.to_datetime( tt_3_app_21_23["date"] ) - dt.datetime( 2019, 4, 21 ) ).dt.total_seconds()

tt_3_app_18_20.fillna( 3*24*3600, inplace = True ) #Quiere decir que no lo realizo
tt_3_app_21_23.fillna( 3*24*3600, inplace = True ) #Quiere decir que no lo realizo

tt_3_app_21_23.to_csv( loc_ftr + "\\tt_3_app_18_20.csv", index = False )
tt_3_app_21_23.to_csv( loc_ftr + "\\tt_3_app_21_23.csv", index = False )

## <span style="color:green"> **Realizo algun evento del TOP 3** </span>
Se corroboro inicialmente que las top3 aplicaciones mas instaladas coinciden en las 3 ventanas

#### <span style="color:orange"> **OneHotEncoding** </span>

In [85]:
top3_app_18_20 = rh_18_20.copy()
top3_app_21_23 = rh_21_23.copy()

cond_1_18_20 = ins_18_20['application_id'] == ins_18_20['application_id'].value_counts().head().index[0]; cond_1_21_23 = ins_21_23['application_id'] == ins_21_23['application_id'].value_counts().head().index[0];
cond_2_18_20 = ins_18_20['application_id'] == ins_18_20['application_id'].value_counts().head().index[1]; cond_2_21_23 = ins_21_23['application_id'] == ins_21_23['application_id'].value_counts().head().index[1];
cond_3_18_20 = ins_18_20['application_id'] == ins_18_20['application_id'].value_counts().head().index[2]; cond_3_21_23 = ins_21_23['application_id'] == ins_21_23['application_id'].value_counts().head().index[2];

m1 = ins_18_20.loc[ cond_1_18_20 | cond_2_18_20 | cond_3_18_20, ["ref_hash", "application_id"] ]; m1["application_id"] = m1["application_id"].astype( "category" )
m2 = ins_21_23.loc[ cond_1_21_23 | cond_2_21_23 | cond_3_21_23, ["ref_hash", "application_id"] ]; m2["application_id"] = m2["application_id"].astype( "category" )

m1 = m1.groupby( "ref_hash" ).agg( {"application_id":"value_counts"} ).unstack( level = 1 )
m2 = m2.groupby( "ref_hash" ).agg( {"application_id":"value_counts"} ).unstack( level = 1 )

top3_app_18_20 = tt_3_app_18_20.merge( m1, how = "left", on = "ref_hash" ); top3_app_18_20.columns = ["ref_hash", "app1", "app2", "app3" ]
top3_app_21_23 = tt_3_app_21_23.merge( m2, how = "left", on = "ref_hash" ); top3_app_21_23.columns = ["ref_hash", "app1", "app2", "app3" ]

top3_app_18_20["app1"] = top3_app_18_20["app1"].apply( lambda x: 1 if x > 0 else 0 ); top3_app_21_23["app1"] = top3_app_21_23["app1"].apply( lambda x: 1 if x > 0 else 0 );
top3_app_18_20["app2"] = top3_app_18_20["app2"].apply( lambda x: 1 if x > 0 else 0 ); top3_app_21_23["app2"] = top3_app_21_23["app1"].apply( lambda x: 1 if x > 0 else 0 );
top3_app_18_20["app3"] = top3_app_18_20["app3"].apply( lambda x: 1 if x > 0 else 0 ); top3_app_21_23["app3"] = top3_app_21_23["app3"].apply( lambda x: 1 if x > 0 else 0 );

top3_app_18_20.to_csv( loc_ftr + "\\top3_app_18_20.csv", index = False )
top3_app_21_23.to_csv( loc_ftr + "\\top3_app_21_23.csv", index = False )

## <span style="color:green"> **Periodo del dia en que se realizo la primera instalacion** </span>

In [199]:
dayp_ins_18_20 = ins_18_20_first_ins[ ["ref_hash", "date"] ]; dayp_ins_18_20["date"] =  pd.to_datetime( dayp_ins_18_20["date"] ).dt.hour
dayp_ins_21_23 = ins_21_23_first_ins[ ["ref_hash", "date"] ]; dayp_ins_21_23["date"] =  pd.to_datetime( dayp_ins_21_23["date"] ).dt.hour

def det_period( x ):
    if (x>=4) & (x<=10): return "mor"
    elif (x>=11) & (x<=15): return "mid"
    elif (x>=16) & (x<=20): return "aft"
    else: return "ngh"

dayp_ins_18_20["period"] = dayp_ins_18_20["date"].apply( lambda x: det_period(x) ); del( dayp_ins_18_20["date"] )
dayp_ins_21_23["period"] = dayp_ins_21_23["date"].apply( lambda x: det_period(x) ); del( dayp_ins_21_23["date"] )

#### <span style="color:orange"> **Mean Encoding** </span> - (Usando el SC)

In [ ]:
X = dayp_ins_18_20[ ["period"] ]; Y = tg_18_20
te = ce.TargetEncoder(smoothing=True); te.fit( X, Y );

In [121]:
dayp_ins_18_20["period_enc"] = te.transform( dayp_ins_18_20[ ["period"] ] ); del( dayp_ins_18_20["period"] )
dayp_ins_21_23["period_enc"] = te.transform( dayp_ins_21_23[ ["period"] ] ); del( dayp_ins_21_23["period"] )

In [123]:
dayp_ins_18_20.to_csv( loc_ftr + "\\dayp_ins_18_20.csv", index = False )
dayp_ins_21_23.to_csv( loc_ftr + "\\dayp_ins_21_23.csv", index = False )

#### <span style="color:orange"> **Mean Encoding** </span> - (Usando la cantidad de instalaciones en ese periodo en esa ventana)

In [187]:
total_ins_period_18_20 = ins_18_20[ ["ref_hash", "date"] ]; total_ins_period_18_20["date"] =  pd.to_datetime( total_ins_period_18_20["date"] ).dt.hour
total_ins_period_21_23 = ins_21_23[ ["ref_hash", "date"] ]; total_ins_period_21_23["date"] =  pd.to_datetime( total_ins_period_21_23["date"] ).dt.hour

total_ins_period_18_20["period"] = total_ins_period_18_20["date"].apply( lambda x: det_period(x) ); del( total_ins_period_18_20["date"] )
total_ins_period_21_23["period"] = total_ins_period_21_23["date"].apply( lambda x: det_period(x) ); del( total_ins_period_21_23["date"] )

In [188]:
total_ins_period_18_20 = total_ins_period_18_20["period"].value_counts().to_frame().reset_index().rename( columns = {"index":"period", "period":"period_count"} )
total_ins_period_21_23 = total_ins_period_21_23["period"].value_counts().to_frame().reset_index().rename( columns = {"index":"period", "period":"period_count"} )

In [189]:
dayp_ins_18_20 = dayp_ins_18_20.merge( total_ins_period_18_20, how = "left", on = "period" ); del( dayp_ins_18_20["period"] )
dayp_ins_21_23 = dayp_ins_21_23.merge( total_ins_period_21_23, how = "left", on = "period" ); del( dayp_ins_21_23["period"] )

In [190]:
dayp_ins_18_20.to_csv( loc_ftr + "\\dayp_ins_18_20.csv", index = False )
dayp_ins_21_23.to_csv( loc_ftr + "\\dayp_ins_21_23.csv", index = False )

#### <span style="color:orange"> **Mean Encoding** </span> - (Usando la cantidad de primeras instalaciones en ese periodo en esa ventana)

In [200]:
period_frt_ins_18_20 = dayp_ins_18_20["period"].value_counts().to_frame().reset_index().rename( columns = {"index":"period", "period":"period_count"} )
period_frt_ins_21_23 = dayp_ins_21_23["period"].value_counts().to_frame().reset_index().rename( columns = {"index":"period", "period":"period_count"} )

In [201]:
dayp_ins_18_20 = dayp_ins_18_20.merge( period_frt_ins_18_20, how = "left", on = "period" ); del( dayp_ins_18_20["period"] )
dayp_ins_21_23 = dayp_ins_21_23.merge( period_frt_ins_21_23, how = "left", on = "period" ); del( dayp_ins_21_23["period"] )

In [202]:
dayp_ins_18_20.to_csv( loc_ftr + "\\dayp_ins_18_20.csv", index = False )
dayp_ins_21_23.to_csv( loc_ftr + "\\dayp_ins_21_23.csv", index = False )